In [1]:
from math import log2
import json

In [2]:
#wczytanie danych
def wczytajDane(nazwa_pliku):
    with open(nazwa_pliku, 'r') as f:
        element = [line.strip() for line in f]
    
    lista = [el.split(',') for el in element]
    
    return lista

In [3]:
lista = wczytajDane("cw 1/test2.txt")
lista

[['0', '0', '0', '0', '1'],
 ['0', '1', '0', '2', '1'],
 ['1', '0', '1', '2', '1'],
 ['1', '0', '1', '1', '1'],
 ['1', '0', '0', '2', '1'],
 ['0', '1', '0', '0', '0'],
 ['1', '1', '0', '0', '0'],
 ['0', '0', '1', '1', '0'],
 ['1', '1', '1', '0', '0'],
 ['1', '0', '1', '0', '0']]

In [4]:
liczbaAtrybutów = len(lista[0])-1
liczbaAtrybutów

4

In [5]:
#lista atrybutów (można ew jak wczytac z pliku)
labels=[]
def nazwy(dane):
    labels = ["A"+str(nr+1) for nr in range(len(dane[0])-1)]
    return labels

labels = nazwy(lista)

In [6]:
liczbaWierszy = len(lista)
liczbaWierszy

10

In [7]:
#ostatnia kolumna - decyzje
decyzje = [wiersz[liczbaAtrybutów] for wiersz in lista]
decyzje

['1', '1', '1', '1', '1', '0', '0', '0', '0', '0']

In [8]:
#ostatnia kolumna - decyzja dla zmiennych danych
def decyzjeKolumna(dane):
    return [k[-1] for k in dane]

In [9]:
decyzjeKolumna(lista)

['1', '1', '1', '1', '1', '0', '0', '0', '0', '0']

In [10]:
#dla każdej unikalnej wartosci w kolumnie zliczam wystąpienia
def licznikUnikalnychWartosci(nrKolumny,dane):
    unikalne = {}
    for wiersz in dane:
        wartosc = wiersz[nrKolumny]
        if wartosc not in unikalne:
            unikalne[wartosc] = 0
        unikalne[wartosc] +=1
    
    return unikalne

In [11]:
for nrK in range(liczbaAtrybutów):
    print(licznikUnikalnychWartosci(nrK,lista))

{'0': 4, '1': 6}
{'0': 6, '1': 4}
{'0': 5, '1': 5}
{'0': 5, '2': 3, '1': 2}


In [12]:
#ile jest unikalnych wartosci w kolumnie
def ileWartWKolumnie(nrKolumny):
    klucze = list(licznikUnikalnychWartosci(nrKolumny,lista).keys())
    licznik = len(klucze)
    
    return licznik

In [13]:
for nrK in range(liczbaAtrybutów):
    s1 ={}
    s1[nrK] = ileWartWKolumnie(nrK)
    print(s1)

{0: 2}
{1: 2}
{2: 2}
{3: 3}


In [14]:
#jakie sa unikalne wartosci w kolumnie
def unikalneWartosciWKolumnie(nrKolumny,dane):
    kolumna = [wiersz[nrKolumny] for wiersz in dane]
    unikalne = set([wartosc for wartosc in kolumna])
    return list(unikalne)

In [15]:
for nrK in range(liczbaAtrybutów):
    print(unikalneWartosciWKolumnie(nrK,lista))

['0', '1']
['0', '1']
['0', '1']
['0', '2', '1']


In [16]:
# tworzy słownik z unikalnymi wartosciami i padajacymi po nich decyzjami
def wartosciDecyzji(dane, k):   # k to kolumna[]
    dlaKolumny = {}
    for ipk,pk in enumerate(k):
        tab = [decyzjeKolumna(dane)[iel] for iel,el in enumerate(k) if el==pk ] #jakie sa decyzje dla wartosci
        #print(tab)
        dlaKolumny[pk] = {wart : tab.count(wart) for wart in decyzjeKolumna(dane)} #for a in tab jakby nie działo
    return dlaKolumny

In [17]:
k = [wiersz[3] for wiersz in lista]
k

['0', '2', '2', '1', '2', '0', '0', '1', '0', '0']

In [18]:
wartosciDecyzji(lista, k)

{'0': {'1': 1, '0': 4}, '2': {'1': 3, '0': 0}, '1': {'1': 1, '0': 1}}

In [19]:
mA = 0
T= liczbaWierszy
InfoCale = {}
GainCale = {}
SplitCale = {}
GainRatioCale = {}

In [20]:
# prawdopodobieństwo
def P(licznikWart): #prametr: słownik licznikUnikalnychWartosci
    p = []
    suma = sum(licznikWart.values())
    for i in licznikWart.keys():
        prawd = licznikWart[i]/suma
        p.append(prawd)
    return p

In [21]:
for nr in range(len(lista[0])-1):
    print(P(licznikUnikalnychWartosci(nr,lista)))

[0.4, 0.6]
[0.6, 0.4]
[0.5, 0.5]
[0.5, 0.3, 0.2]


In [22]:
#entropia
def I(P):
    wynik = (-1)*sum([p*log2(p) for p in P if p!=0])
    return wynik

In [23]:
podsumowanieDecyzji = licznikUnikalnychWartosci(liczbaAtrybutów,lista) # unikalne decyzje+licznik
print(podsumowanieDecyzji)
zbior = [podsumowanieDecyzji[a]/len(decyzje) for a in podsumowanieDecyzji] #prawdopodobieństo dla całego zbioru
print(zbior)

{'1': 5, '0': 5}
[0.5, 0.5]


In [24]:
def Informacje(dane):
    
    Info = []
    Gain = []
    Split = []
    GainRatio = []
    
    liczbaAtrybutów =  len(dane[0])-1
    print(len(dane))
    
    for numer in range(liczbaAtrybutów):  #el=numer
        
        info=0
        split=0
        gain=0
        gainRatio=0       
        
        kolumna = [wiersz[numer] for wiersz in dane] #cala kolumna
        atr = licznikUnikalnychWartosci(numer,dane)
        T = sum(atr.values())
        
        print("\n")
        print("A", numer+1)
        print(atr) 
        print(T)
        
        p = {}    
        p = wartosciDecyzji(dane, kolumna)
        
        for k in sorted(p.keys()):
            print("\n")
            print("\t", "Decyzje")
            print("\t", k) 
            dec = p[k]
            Ti = sum(dec.values())
            PP = P(dec)
            print("\t", dec)
            print("\t", PP)
            print("\t", I(PP),'*', Ti/T)

            info += (Ti/T * I(PP))
            split += ((-1)*(Ti/T)* log2(Ti/T))

        gain = I(P(licznikUnikalnychWartosci(liczbaAtrybutów,dane))) - info
        
        if split > 0:
            gainRatio = gain/split

        Info.append(info)
        Gain.append(gain)
        Split.append(split)
        GainRatio.append(gainRatio)
        
        # do tabeli Info/Gain/Split/GainRatio
        InfoCale[numer] = info
        GainCale[numer] = gain
        SplitCale[numer] = split
        GainRatioCale[numer] = gainRatio
        
    maxA=Gain.index(max(Gain))
    maxGain = max(Gain)
    
    #do tabeli Info/Gain/Split/GainRatio
    mA=maxA
    
    return maxA, maxGain


In [25]:
print("=====")
print(podsumowanieDecyzji)
print(zbior)
print("Entropia:",I(zbior)) # etropia dla całego zbioru
print("=====")


Informacje(lista)

=====
{'1': 5, '0': 5}
[0.5, 0.5]
Entropia: 1.0
=====
10


A 1
{'0': 4, '1': 6}
10


	 Decyzje
	 0
	 {'1': 2, '0': 2}
	 [0.5, 0.5]
	 1.0 * 0.4


	 Decyzje
	 1
	 {'1': 3, '0': 3}
	 [0.5, 0.5]
	 1.0 * 0.6


A 2
{'0': 6, '1': 4}
10


	 Decyzje
	 0
	 {'1': 4, '0': 2}
	 [0.6666666666666666, 0.3333333333333333]
	 0.9182958340544896 * 0.6


	 Decyzje
	 1
	 {'1': 1, '0': 3}
	 [0.25, 0.75]
	 0.8112781244591328 * 0.4


A 3
{'0': 5, '1': 5}
10


	 Decyzje
	 0
	 {'1': 3, '0': 2}
	 [0.6, 0.4]
	 0.9709505944546686 * 0.5


	 Decyzje
	 1
	 {'1': 2, '0': 3}
	 [0.4, 0.6]
	 0.9709505944546686 * 0.5


A 4
{'0': 5, '2': 3, '1': 2}
10


	 Decyzje
	 0
	 {'1': 1, '0': 4}
	 [0.2, 0.8]
	 0.7219280948873623 * 0.5


	 Decyzje
	 1
	 {'1': 1, '0': 1}
	 [0.5, 0.5]
	 1.0 * 0.2


	 Decyzje
	 2
	 {'1': 3, '0': 0}
	 [1.0, 0.0]
	 -0.0 * 0.3


(3, 0.4390359525563188)

In [26]:
print("\n")
for nr in range(liczbaAtrybutów):
    print(f"Info({labels[nr]},T)= {InfoCale[nr]}   \t Gain({labels[nr]},T)= {GainCale[nr]}")

#wyznacza max Gain
#maxA=Gain.index(max(Gain))

print(f"\nWybrany atrybut {labels[mA]}, bo w tym przypadku najwyższa jest wartość Gain: {max(GainCale.values())}")

print("\n")
for nr in range(liczbaAtrybutów): 
    print(f"Split(D{str(nr+1)},T)= {SplitCale[nr]}   \t Gain Ratio(D{str(nr+1)},T)= {GainRatioCale[nr]}")



Info(A1,T)= 1.0   	 Gain(A1,T)= 0.0
Info(A2,T)= 0.8754887502163469   	 Gain(A2,T)= 0.12451124978365313
Info(A3,T)= 0.9709505944546686   	 Gain(A3,T)= 0.02904940554533142
Info(A4,T)= 0.5609640474436812   	 Gain(A4,T)= 0.4390359525563188

Wybrany atrybut A1, bo w tym przypadku najwyższa jest wartość Gain: 0.4390359525563188


Split(D1,T)= 0.9709505944546686   	 Gain Ratio(D1,T)= 0.0
Split(D2,T)= 0.9709505944546686   	 Gain Ratio(D2,T)= 0.12823644219877584
Split(D3,T)= 1.0   	 Gain Ratio(D3,T)= 0.02904940554533142
Split(D4,T)= 1.4854752972273344   	 Gain Ratio(D4,T)= 0.29555250994465343


In [27]:
listaNumerow = [nr for nr in range(len(lista[0])-1)]
listaNumerow

[0, 1, 2, 3]

In [28]:
def nowaLista(dane, atryb, w): #atryb = lista atryb
    nowaL = []
    for wiersz in dane:
        if wiersz[atryb] == w:
            e = wiersz[:atryb]        #kopia atrybutow
            e.extend(wiersz[atryb:]) #dodaje do listy wszystkie el z listy
            nowaL.append(e)     #dodaje do nowej listy
    return nowaL

In [29]:
def licznikDecyzji(d):
    licznik = {}
    for decyzja in d:
        if decyzja not in licznik.keys():
            decyzje[decyzja] = 0
        licznik[decyzja] += 1
    return max(licznik)

In [37]:
def szukaj(dane, atrybuty):
    #print("\nNajlepszy atrybut: ", labels[atryb])
    
    maxA, maxGain = Informacje(dane) #najlepszy atrybut, wart max Gain
    
    d = decyzjeKolumna(dane)  #kolumna decyzji

    #wyswietla decyzje, koniec podziału
    if d.count(d[0]) == len(d):
        return(d[0])
        
    if len(dane[0]) == 1:
        return licznikDecyzji(d)
    
    if maxGain < 0 or maxGain == 0: #samo ==
        return licznikDecyzji(d)
       
    atrybut = atrybuty[maxA] #atrybuty
    drzewo = { atrybut: {} }
    #del[atrybuty[maxA]]
    unik = unikalneWartosciWKolumnie(maxA,dane)
    
    for w in unik:
        noweA = atrybuty[:]
        drzewo[atrybut][w] = szukaj(nowaLista(dane, maxA, w), noweA)    
      
    return drzewo

In [38]:
def budujDrzewo(dane):
    print("\nDrzwo decyzyjne dla {0} przypadków ({1} atrybutów)".format(liczbaWierszy,liczbaAtrybutów))
    d=wczytajDane(dane)
    koniec = szukaj(d, nazwy(d)) 
    print(json.dumps(koniec, indent=4, sort_keys=True))

In [39]:
budujDrzewo('cw 1/test2.txt')


Drzwo decyzyjne dla 10 przypadków (4 atrybutów)
10


A 1
{'0': 4, '1': 6}
10


	 Decyzje
	 0
	 {'1': 2, '0': 2}
	 [0.5, 0.5]
	 1.0 * 0.4


	 Decyzje
	 1
	 {'1': 3, '0': 3}
	 [0.5, 0.5]
	 1.0 * 0.6


A 2
{'0': 6, '1': 4}
10


	 Decyzje
	 0
	 {'1': 4, '0': 2}
	 [0.6666666666666666, 0.3333333333333333]
	 0.9182958340544896 * 0.6


	 Decyzje
	 1
	 {'1': 1, '0': 3}
	 [0.25, 0.75]
	 0.8112781244591328 * 0.4


A 3
{'0': 5, '1': 5}
10


	 Decyzje
	 0
	 {'1': 3, '0': 2}
	 [0.6, 0.4]
	 0.9709505944546686 * 0.5


	 Decyzje
	 1
	 {'1': 2, '0': 3}
	 [0.4, 0.6]
	 0.9709505944546686 * 0.5


A 4
{'0': 5, '2': 3, '1': 2}
10


	 Decyzje
	 0
	 {'1': 1, '0': 4}
	 [0.2, 0.8]
	 0.7219280948873623 * 0.5


	 Decyzje
	 1
	 {'1': 1, '0': 1}
	 [0.5, 0.5]
	 1.0 * 0.2


	 Decyzje
	 2
	 {'1': 3, '0': 0}
	 [1.0, 0.0]
	 -0.0 * 0.3
5


A 1
{'0': 2, '1': 3}
5


	 Decyzje
	 0
	 {'1': 1, '0': 1}
	 [0.5, 0.5]
	 1.0 * 0.4


	 Decyzje
	 1
	 {'1': 0, '0': 3}
	 [0.0, 1.0]
	 -0.0 * 0.6


A 2
{'0': 2, '1': 3}
5


	 Decyzje
	 0

In [40]:
budujDrzewo('cw 1/zoo.txt')


Drzwo decyzyjne dla 10 przypadków (4 atrybutów)
101


A 1
{'aardvark': 1, 'antelope': 1, 'bass': 1, 'bear': 1, 'boar': 1, 'buffalo': 1, 'calf': 1, 'carp': 1, 'catfish': 1, 'cavy': 1, 'cheetah': 1, 'chicken': 1, 'chub': 1, 'clam': 1, 'crab': 1, 'crayfish': 1, 'crow': 1, 'deer': 1, 'dogfish': 1, 'dolphin': 1, 'dove': 1, 'duck': 1, 'elephant': 1, 'flamingo': 1, 'flea': 1, 'frog': 2, 'fruitbat': 1, 'giraffe': 1, 'girl': 1, 'gnat': 1, 'goat': 1, 'gorilla': 1, 'gull': 1, 'haddock': 1, 'hamster': 1, 'hare': 1, 'hawk': 1, 'herring': 1, 'honeybee': 1, 'housefly': 1, 'kiwi': 1, 'ladybird': 1, 'lark': 1, 'leopard': 1, 'lion': 1, 'lobster': 1, 'lynx': 1, 'mink': 1, 'mole': 1, 'mongoose': 1, 'moth': 1, 'newt': 1, 'octopus': 1, 'opossum': 1, 'oryx': 1, 'ostrich': 1, 'parakeet': 1, 'penguin': 1, 'pheasant': 1, 'pike': 1, 'piranha': 1, 'pitviper': 1, 'platypus': 1, 'polecat': 1, 'pony': 1, 'porpoise': 1, 'puma': 1, 'pussycat': 1, 'raccoon': 1, 'reindeer': 1, 'rhea': 1, 'scorpion': 1, 'seahorse': 1, '



	 Decyzje
	 0
	 {'1': 40, '4': 0, '2': 0, '7': 1, '6': 0, '5': 0, '3': 1}
	 [0.9523809523809523, 0.0, 0.0, 0.023809523809523808, 0.0, 0.0, 0.023809523809523808]
	 0.3238144752669867 * 0.4158415841584158


	 Decyzje
	 1
	 {'1': 1, '4': 13, '2': 20, '7': 9, '6': 8, '5': 4, '3': 4}
	 [0.01694915254237288, 0.22033898305084745, 0.3389830508474576, 0.15254237288135594, 0.13559322033898305, 0.06779661016949153, 0.06779661016949153]
	 2.440717570606549 * 0.5841584158415841


A 5
{'1': 41, '0': 60}
101


	 Decyzje
	 0
	 {'1': 0, '4': 13, '2': 20, '7': 10, '6': 8, '5': 4, '3': 5}
	 [0.0, 0.21666666666666667, 0.3333333333333333, 0.16666666666666666, 0.13333333333333333, 0.06666666666666667, 0.08333333333333333]
	 2.3840039346603206 * 0.594059405940594


	 Decyzje
	 1
	 {'1': 41, '4': 0, '2': 0, '7': 0, '6': 0, '5': 0, '3': 0}
	 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	 -0.0 * 0.40594059405940597


A 6
{'0': 77, '1': 24}
101


	 Decyzje
	 0
	 {'1': 39, '4': 13, '2': 4, '7': 10, '6': 2, '5': 4, '3': 

{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0
1


A 1
{'bass': 1}
1


	 Decyzje
	 bass
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 2
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'1': 1}
1


	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0
1


A 1
{'seal': 1}
1


	 Decyzje
	 seal
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 2
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}




A 1
{'hare': 1}
1


	 Decyzje
	 hare
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 2
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'4': 1}
1


	 Decyzje
	 4
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 16
{'0': 

	 carp
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 2
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 16
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}


	 -0.0 * 1.0


A 2
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'4': 1}
1


	 Decyzje
	 4
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 16
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 1

	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'1': 1}
1


	 Decyzje
	 1
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 16
{'0': 1}
1


	 Decyzje
	 0
	 {'4': 1}
	 [1.0]
	 -0.0 * 1.0


A 17
{'0': 1}
1


	 Decyzje
	 0
	 {

A 2
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'2': 1}
1


	 Decyzje
	 2
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'1': 1}
1


	 Decyzje
	 1
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 16
{'0': 1}
1


	 Decyzje
	 0
	 {'1': 1}
	 [1.0]
	 -0.0 * 1.0


A 17
{'0': 1}
1




	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 3
{'1': 1}
1


	 Decyzje
	 1
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 4
{'1': 1}
1


	 Decyzje
	 1
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 5
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 6
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 7
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 8
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 9
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 10
{'1': 1}
1


	 Decyzje
	 1
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 11
{'1': 1}
1


	 Decyzje
	 1
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 12
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 13
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 14
{'2': 1}
1


	 Decyzje
	 2
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 15
{'1': 1}
1


	 Decyzje
	 1
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 16
{'0': 1}
1


	 Decyzje
	 0
	 {'2': 1}
	 [1.0]
	 -0.0 * 1.0


A 17
{'1': 1}
1


	 Decyzje
	 1
	 {'2': 1}
	 

In [41]:
dane = wczytajDane("cw 1/test2.txt")
koniec = szukaj(dane, nazwy(dane)) 

10


A 1
{'0': 4, '1': 6}
10


	 Decyzje
	 0
	 {'1': 2, '0': 2}
	 [0.5, 0.5]
	 1.0 * 0.4


	 Decyzje
	 1
	 {'1': 3, '0': 3}
	 [0.5, 0.5]
	 1.0 * 0.6


A 2
{'0': 6, '1': 4}
10


	 Decyzje
	 0
	 {'1': 4, '0': 2}
	 [0.6666666666666666, 0.3333333333333333]
	 0.9182958340544896 * 0.6


	 Decyzje
	 1
	 {'1': 1, '0': 3}
	 [0.25, 0.75]
	 0.8112781244591328 * 0.4


A 3
{'0': 5, '1': 5}
10


	 Decyzje
	 0
	 {'1': 3, '0': 2}
	 [0.6, 0.4]
	 0.9709505944546686 * 0.5


	 Decyzje
	 1
	 {'1': 2, '0': 3}
	 [0.4, 0.6]
	 0.9709505944546686 * 0.5


A 4
{'0': 5, '2': 3, '1': 2}
10


	 Decyzje
	 0
	 {'1': 1, '0': 4}
	 [0.2, 0.8]
	 0.7219280948873623 * 0.5


	 Decyzje
	 1
	 {'1': 1, '0': 1}
	 [0.5, 0.5]
	 1.0 * 0.2


	 Decyzje
	 2
	 {'1': 3, '0': 0}
	 [1.0, 0.0]
	 -0.0 * 0.3
5


A 1
{'0': 2, '1': 3}
5


	 Decyzje
	 0
	 {'1': 1, '0': 1}
	 [0.5, 0.5]
	 1.0 * 0.4


	 Decyzje
	 1
	 {'1': 0, '0': 3}
	 [0.0, 1.0]
	 -0.0 * 0.6


A 2
{'0': 2, '1': 3}
5


	 Decyzje
	 0
	 {'1': 1, '0': 1}
	 [0.5, 0.5]
	 1.0 * 0.4


	 

In [35]:
print(koniec)

{'A4': {'0': {'A1': {'0': {'A2': {'0': '1', '1': '0'}}, '1': '0'}}, '2': '1', '1': {'A1': {'0': '0', '1': '1'}}}}


In [42]:
print(json.dumps(koniec, indent=4, separators=(",",  " = "), sort_keys=True))

{
    "A4" = {
        "0" = {
            "A1" = {
                "0" = {
                    "A2" = {
                        "0" = "1",
                        "1" = "0"
                    }
                },
                "1" = "0"
            }
        },
        "1" = {
            "A1" = {
                "0" = "0",
                "1" = "1"
            }
        },
        "2" = "1"
    }
}
